In [1]:
import re
import numpy as np
import pandas as pd
import xlwt
import os

In [1]:
#Read in Excel
df = pd.read_excel("../../Databases/HPIDB2.0/Excel files/Shigella flexneri.xlsx", names=["Human_protein","Pathogen_protein","Detection_method", "Pubmed_ID","Taxid_B","Taxid_Pathogen","Interaction_type","Interaction_ID"], usecols="A,B,G,I:L,N", header=0)

#Rearrange columns
columnsTitles=["Taxid_Pathogen","Pathogen_protein","Taxid_B","Protein_B","Detection_method", "Pubmed_ID","Interaction_type","Interaction_ID"]
df=df.reindex(columns=columnsTitles)

NameError: name 'pd' is not defined

In [9]:
#Determine source database protein_ID after merge
A=df.Protein_B.apply(lambda x: x.split(":")[0])
B=df.Pathogen_protein.apply(lambda y: y.split(":")[0])
C=[]
for i in range(len(A)):
    if A[i]== B[i]:
        C.append(A[i])
    else:
        C.append((A[i],B[i]))
df= df.assign(Database=C)

In [10]:
#Extract Interaction_ID after merge
Int=df["Interaction_ID"]
Int = Int.str.extract(r'(?:EBI-)([0-9]*)',expand =True)
df["Interaction_ID"]=Int

In [11]:
#Extract protein IDs and rearrange columns dataframe after merge
H_p = df["Protein_B"]
H_p=H_p.str.replace(r'(?:uniprotkb:)([a-zA-Z0-9]*)', '\g<1>')
df["Protein_B"]=H_p

P_p=df["Pathogen_protein"]
P_p=P_p.str.replace(r'(?:uniprotkb:)([a-zA-Z0-9]*)','\g<1>')
df["Pathogen_protein"]=P_p

In [12]:
#Extract detection method after merge
D_m = df["Detection_method"]
D_m=D_m.str.replace((r'(?:psi-mi:MI:[0-9]*[(])([^)]*)(?:[)])'), '\g<1>')
df["Detection_method"]=D_m

In [13]:
#Extract pubmed_ID after merge
P_ID=df["Pubmed_ID"]
P_ID=P_ID.str.replace(r'(?:[^p]*)(?:pubmed:)([0-9]*)(?:.*)','\g<1>')
df["Pubmed_ID"]=P_ID

In [14]:
#Extract interaction type after merge
I_t = df["Interaction_type"]
I_t=I_t.str.replace((r'(?:psi-mi:MI:[0-9]*[(])([^)]*)(?:[)])'), '\g<1>')
df["Interaction_type"]=I_t

In [15]:
#Rearrange rows of dataframe after merge
df=df.sort_values("Pathogen_protein")
df = df.reset_index(drop=True)

In [16]:
df.to_excel("../Test outputs/Shigella flexneri HPIDB.xls")